# 予測市場におけるマーケットスコアリングルールについて

予測市場よりも遥か以前から[スコアリングルール](http://www.geocities.jp/toyotaka_sakai/2015Nov3rdweek4.pdf)と呼ばれるものが存在した。
これは社会的意思決定理論の産物で、投票内容をどのようにアウトカム（投票結果）にマッピングするかを定義した関数。例えば

- 多数決
- [ボルダルール](https://ja.wikipedia.org/wiki/%E3%83%9C%E3%83%AB%E3%83%80%E5%BE%97%E7%82%B9)
- [Keremy-Young method(コンドルセ-ヤングの最尤法)](https://en.wikipedia.org/wiki/Kemeny%E2%80%93Young_method)

などがある。

しかし単なる投票だと、いずれの場合でも人数が増えるほど[「投票者の無気力化(Voter Apathy)」](https://en.wikipedia.org/wiki/Voter_apathy)
という問題が発生する。

予測市場ならばこの問題を乗り越えられるが、代わりに「単なるオッズ比では人数が少ないほど賭けるインセンティブが少ないため誰も最初の賭けをしたがらない」
という問題がある。
また、原理上「自分が他人よりもよく知っていると思っていること」に対してしか賭けが成立しないため、さらに参加者が少なくなる傾向がある。

マーケットスコアリングルールはこの両者の利点を橋渡しするためのもので、具体的にはコスト関数(あるいはスコア関数)と呼ばれるものを使用する。

Truthcoinでは(従来の多くの予測市場と同様に)LMSRと呼ばれるコスト関数を用いている



# コスト関数とは

コスト関数とは、特定の証券を一定量買うのに必要な資金量を計算するための関数で、$C$で表される。

* $o \in O$ ... 未来に発生する特定の事象(賭けの対象)
* $q_o$ ... 現時点で市場に出回っている$o$の証券($o$が発生したときのみ1ドル、そうでなければ0ドルの価値を持つ)の総数
* $r$ ... $O$次元の「購入ベクトル」、どの事象の証券をどれだけ買うかを表した物

とすると、購入時のコストは

$C(q + r) - C(q)$

## [Logarithmic Market Scoring Rule(LMSR)](http://mason.gmu.edu/~rhanson/mktscore.pdf)について

### 賭ける内容が二択の場合

例えばバイナリのDecisionの場合、以下のコスト関数を用いる。

$$C(q_0, q_1) = b * \ln(e^{q_0/b} + e^{q_1/b})$$

ただし

* $b$ ... Authorが指定する流動性パラメータ。高いほど市場開始時の流動性を担保できる(誰もベットに参加しないことを避けられる)が、Marketの作成にあたってAuthorの払うコストが大きい
* $e$ ... 自然対数の底

例えば0のアウトカムを13単位買いたい場合にかかる費用は$C(q_0+13, q_1)$
1を150単位売りたい場合は$C(q_0, q_1-150)$がその費用となる。

複数次元のDecisionにおいて特定の次元のみを「見て」投票したい場合でも、十分な正確さとIncentive Compatibilityが見込める点にLogを使用するメリットがある。

### 賭ける内容がカテゴリカル（三択以上の場合）

$$C(q) = b \log \sum_{o \in O}e^{q_0/b}$$

ただし、$O$は取りうる全ての状態（アウトカム）で、$o$は購入対象

アウトカム$q_0$を$r$株買う際のコストは
$C(q + r) - C(q) $となる

見てわかるように、コスト関数が購入する対象だけに依存しており、マーケット全体の状態に依存しないので、複雑なマーケットでも計算が楽、あるいは購入時の認知コストが少ない


### 価格関数

特定の時刻におけるアウトカム$o$の証券の価格を$p_o (0 \le p_o \le 1 )$とすると、これは$q_o$の関数であり計算式は価格関数(price function)と呼ばれる。
LMSRの場合、価格関数は以下

$$p_o(q) = \frac{\partial{C(q)}}{\partial{q_o}} = \frac{e^{q_o/b}}{\sum_{o' \in O}e^{q_o'/b}}$$

LMSRは全体で微分可能なので、価格計算においても扱い安いことが普及の一因となっている。

また、$q_o$の合計は1(ドル)になるので、確率分布として扱え、「事象$o$の発生確率に関する現時点での市場の信念」として扱うことができる。

LMSRを用いた場合、マーケット作成者が失う金額の上限は$b\log|O|$となる。

### 代替案

LMSRは予測市場で最も一般的に使用されているもので、たとえば
すでに以下で使用されている。

* [InklingMarkets](http://inklingmarkets.com/)
* [the Washington Stock Exchange](http://www.thewsx.com/)
* [BizPredict](http://bizpredict.com/)
* [YooNew](http://www.yoonew.com/)
* [Net Exchange](http://www.nex.com/)

[流動性パラメータ$b$をダイナミックに変更することで効率性を高めることができるという研究](http://www.cs.cmu.edu/~aothman/flex.pdf)も存在する。

代替案には

* [ベイジアンネットワークを用いたもの](http://www.cs.rpi.edu/~xial/Files/PennockXiaUAI11.pdf) ... LMSRは状態数に対して計算量が多項式時間で増加するので、大量のアウトカムが存在する場合に有用
* [ダイナミックパリミューチュアル](http://dpennock.com/papers/pennock-ec-2004-dynamic-parimutuel.pdf)


などがある。（が、LMSRほどは使われていないようだ。）

# LMSRの描画

二択の場合のコスト関数を描画する

In [5]:
import numpy as np
# for regular plotly ctions
import plotly.plotly as py
import plotly    
import plotly.graph_objs as go

# for widgets
from ipywidgets import interact, interactive

In [55]:
def LMSR(q0, q1, b):
    return b * np.log(np.e ** (q0 / b) + np.e ** (q1 / b))

def cost(q0, q1, b, amount):
    return LMSR(q0 + amount, q1, b) - LMSR(q0, q1, b)

x, y = np.meshgrid(np.linspace(0, 200, 32), np.linspace(0, 200, 32))

In [57]:
# setup plotly
plotly.offline.init_notebook_mode(connected=True)

@interact(b=(0.5, 30.0, 1))
def plot(b=1):
    z = LMSR(x, y, b)
    trace1 = go.Surface(x = x, y = y, z=z, name="Cost function")     
    data=[trace1]
    layout = go.Layout(
        title='Cost function',

        margin=dict(
            l=0,
            r=0,
            b=100,
            t=100
        ),
        xaxis = dict(
            title="amount x bought",
        ),
        yaxis = dict(
            title="amount y bought",
        )
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='Costfunction-3d-surface')

ある事象が起こると「Yes」証券は1ドル、「No」証券は0ドルになり、起こらなければ逆の価値を持つとする。

1. x軸 ... 現時点で市場にある「Yes」証券の量
2. y軸 ... 現時点で市場にある「No」証券の量

b(liquidity係数)を変えると関数がなめらかになっていることがわかる。
先述のように購入にかかるコストはコスト関数から以下の式で直接計算できる。

$$C(q + r) - C(q)$$

描画してみる。こちらの方がわかりやすいかもしれない。


In [58]:

@interact(b=(0.5, 30.0, 1), x_amount_to_buy=(-30, 50, 10))
def plot(b=1, x_amount_to_buy = 10):
    z = LMSR(x + x_amount_to_buy, y, b) - LMSR(x, y, b)
    trace1 = go.Surface(x=x,y=y, z=z)     
    data=[trace1]
    layout = go.Layout(
    title='Cost to buy',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
        )
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='Costfunction-3d-surface')

$b$が大きいほど、入力に対してsensitiveになる

In [60]:
np.gradient??